In [54]:
import copy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def get_ids():
    target_ids = pd.read_csv('data/SampleSubmissionStage1.csv').set_index('ID').drop('Pred', axis=1)
    target_ids['Season'] = target_ids.index.map(lambda i: i[:4])
    target_ids['team_a'] = target_ids.index.map(lambda i: i[5:9])
    target_ids['team_b'] = target_ids.index.map(lambda i: i[10:14])
    target_ids['in_target'] = True
    ncaa_games = pd.read_csv('data/NCAATourneyCompactResults.csv')
    train_ids = pd.DataFrame(ncaa_games.Season.map(str) + '_'
                            + ncaa_games.WTeamID.map(str) + '_'
                            + ncaa_games.LTeamID.map(str))\
        .rename(columns={0: 'ID'}).set_index('ID')
    train_ids['Season'] = train_ids.index.map(lambda i: i[:4])
    train_ids['team_a'] = train_ids.index.map(lambda i: i[5:9])
    train_ids['team_b'] = train_ids.index.map(lambda i: i[10:14])
    train_ids_sort = train_ids
    train_ids_sort['team_a'] = train_ids[['team_a', 'team_b']]\
        .apply(lambda r: r[0] if int(r[0]) > int(r[1]) else r[1], axis=1)
    train_ids_sort['team_b'] = train_ids[['team_a', 'team_b']]\
        .apply(lambda r: r[1] if int(r[0]) > int(r[1]) else r[1], axis=1)
    train_ids[['team_a', 'team_b']] = train_ids_sort[['team_a', 'team_b']]
    return train_ids, target_ids

train, target = get_ids()
train.sample()

,Season,team_a,team_b
ID,,,
2013_1211_1380,2013,1380,1380


In [8]:
data = pd.concat([train, target]).fillna(0)
data.head()

,Season,in_target,team_a,team_b
ID,,,,
1985_1116_1234,1985,0,1234,1234
1985_1120_1345,1985,0,1345,1345
1985_1207_1250,1985,0,1250,1250
1985_1229_1425,1985,0,1425,1425
1985_1242_1325,1985,0,1325,1325


In [9]:
ncaa_games = pd.read_csv('data/NCAATourneyCompactResults.csv')
ncaa_games = ncaa_games.astype({
    'LTeamID': str,
    'WTeamID': str,
    'Season': str
})
ncaa_games['diff'] = ncaa_games['WScore'] - ncaa_games['LScore']
print(ncaa_games.shape)
ncaa_games.sample()

(2117, 9)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,diff
260,1989,136,1257,76,1114,71,N,0,5


In [10]:
data.shape

(11229, 4)

In [11]:
win_teams = pd.merge(data, ncaa_games,
                     left_on=['Season', 'team_a', 'team_b'],
                     right_on=['Season', 'WTeamID', 'LTeamID'], how='left').dropna()
win_teams['a_win'] = 1
data = pd.merge(data, win_teams[['Season', 'team_a', 'a_win']],
                on=['Season', 'team_a'], how='left').fillna(0)
print('A Win %: {:.2f}'.format(data.a_win.mean()))
data.sample()

A Win %: 0.70


,Season,in_target,team_a,team_b,a_win
8759,2015,True,1214,1323,1.0


In [12]:
data = data.drop_duplicates(['Season', 'team_a', 'team_b'])

In [13]:
for l in range(1,4):
    ldiffs = ncaa_games.groupby(['Season', 'LTeamID'])[['diff']].sum()\
        .reset_index().pivot(index='Season', columns='LTeamID', values='diff').fillna(0).shift(l)\
        .reset_index().melt(id_vars='Season').dropna().rename(columns={'value': 'avg_ldiff_{}_y_ago'.format(l)})
    wdiffs = ncaa_games.groupby(['Season', 'WTeamID'])[['diff']].sum()\
        .reset_index().pivot(index='Season', columns='WTeamID', values='diff').fillna(0).shift(l)\
        .reset_index().melt(id_vars='Season').dropna().rename(columns={'value': 'avg_wdiff_{}_y_ago'.format(l)})
    
    data = pd.merge(data, ldiffs,
                    left_on=['Season', 'team_a'], right_on=['Season', 'LTeamID'],
                    how='left').drop('LTeamID', axis=1)
    data = pd.merge(data, wdiffs,
                    left_on=['Season', 'team_a'], right_on=['Season', 'WTeamID'],
                    how='left').drop('WTeamID', axis=1)
    data = pd.merge(data, ldiffs,
                    left_on=['Season', 'team_b'], right_on=['Season', 'LTeamID'],
                    how='left', suffixes=['','_b']).drop('LTeamID', axis=1)
    data = pd.merge(data, wdiffs,
                    left_on=['Season', 'team_b'], right_on=['Season', 'WTeamID'],
                    how='left', suffixes=['','_b']).drop('WTeamID', axis=1)
    data = data.dropna()
    
data.sample()

,Season,in_target,team_a,team_b,a_win,avg_ldiff_1_y_ago,avg_wdiff_1_y_ago,avg_ldiff_1_y_ago_b,avg_wdiff_1_y_ago_b,avg_ldiff_2_y_ago,avg_wdiff_2_y_ago,avg_ldiff_2_y_ago_b,avg_wdiff_2_y_ago_b,avg_ldiff_3_y_ago,avg_wdiff_3_y_ago,avg_ldiff_3_y_ago_b,avg_wdiff_3_y_ago_b
6180,2016,True,1242,1372,1.0,13.0,19.0,7.0,0.0,3.0,11.0,17.0,2.0,2.0,19.0,0.0,0.0


In [28]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss


X = data[~data.in_target.astype(bool)].drop(['Season', 'team_a', 'team_b'], axis=1)
y = data[~data.in_target.astype(bool)][['a_win']]

kf = KFold(n_splits=5, shuffle=True)
for tr_i, t_i in kf.split(X):
    X_tr, y_tr = X.iloc[tr_i], y.iloc[tr_i]
    X_t, y_t = X.iloc[t_i], y.iloc[t_i]
    knn = KNeighborsClassifier(n_neighbors=15)
    knn.fit(X_tr.values, y_tr.values.reshape(-1))
    preds = knn.predict_proba(X_t.values)
    print(log_loss(y_t.values, preds))

0.3244546613239674
0.293577276708752
0.32423671287460737
0.2950053989885576
0.33944602319738454


In [53]:
X_sub = pd.merge(target, data, how='left').drop(['Season', 'team_a', 'team_b'], axis=1).fillna(0)
y_sub = pd.merge(target, data, how='left')[['Season', 'team_a', 'team_b']].fillna(0) 

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X.values, y.values.reshape(-1))
preds = knn.predict_proba(X_sub.values)
preds

array([[1.        , 0.        ],
       [0.        , 1.        ],
       [0.13333333, 0.86666667],
       ...,
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ]])

In [57]:
sub = cp.deepcopy(y_sub)
sub['Pred'] = preds[:,1]
sub['ID'] = sub['Season'].map(str) + '_' + sub['team_a'].map(str) + '_' + sub['team_b'].map(str)
sub.set_index('ID', inplace=True)
sub.drop(['Season', 'team_a', 'team_b'], axis=1, inplace=True)
sub.head()

,Pred
ID,
2014_1107_1110,0.000000
2014_1107_1112,1.000000
2014_1107_1113,0.866667
2014_1107_1124,1.000000
2014_1107_1140,1.000000
